# CS486 - Artificial Intelligence
## Lesson 21 - Probability

Today we will discuss probabilistic inference. Our goal is to learn something about *random variables* that we care about through the observation of some other set of variables. We'll explore these concepts using some AIMA helpers. 

In [ ]:
import helpers
from aima.probability import *

## Distributions

A **random variable** is a variable whose possible values are outcomes of a random phenomenon. In other words their values aren't described by a function, but by a **probability distribution**. 

Consider the example of a friend who randomly brings an umbrella with her to work. Over two weeks, she brought her umbrella on 7 days. We cannot write a function that tells us when she'll have the umbrella, but we can write the distribution $P(Umbrella)$:

| Umbrella | P |
|---|---|
| `True` | 7 | 
| `False` | 7 |

A **normal distribution** is one whose values sum to 1. You can normalize a distribution by dividing each value by the sum of all the values. For the example above, we can normalize by dividing by 14. AIMA's `ProbDist` class can also normalize values: 

In [ ]:
umbrella = ProbDist('Umbrella', {True: 7, False: 7})
umbrella.normalize()
umbrella.show_approx()

A **joint distribution** is a distribution that describes the probabilities of outcomes involving two or more variables. For example, consider the joint distribution of whether or not it is raining when our friend has her umbrella:

| Forecast | Rain | Umbrella | P |
|-------|-------|-------|---|
| True  | True  | True  | 0.25 | 
| True  | True  | False | 0.09 |
| True  | False | True  | 0.14 |
| True  | False | False | 0.08 |
| False | True  | True  | 0.07 |
| False | True  | False | 0.16 |
| False | False | True  | 0.04 |
| False | False | False | 0.17 |

In [ ]:
joint_dist = JointProbDist(['Forecast','Rain','Umbrella'])
joint_dist[dict(Forecast=True,  Rain=True,  Umbrella=True)]  = 0.25
joint_dist[dict(Forecast=True,  Rain=True,  Umbrella=False)] = 0.09
joint_dist[dict(Forecast=True,  Rain=False, Umbrella=True)]  = 0.14
joint_dist[dict(Forecast=True,  Rain=False, Umbrella=False)] = 0.08
joint_dist[dict(Forecast=False, Rain=True,  Umbrella=True)]  = 0.07
joint_dist[dict(Forecast=False, Rain=True,  Umbrella=False)] = 0.16
joint_dist[dict(Forecast=False, Rain=False, Umbrella=True)]  = 0.04
joint_dist[dict(Forecast=False, Rain=False, Umbrella=False)] = 0.17

So have the odds of our friend carrying her umbrella changed? Well, we can compute the **marginal distribution** for `Umbrella`. It is called *marginal* because if you were to do this by hand, you would find all the entries where the `Umbrella` values were the same and sum them *in the margins*. Mathematically, a marginal distribution is: 

$$P(X=x)=\sum\limits_y P(X=x,Y=y)$$ 

Given the joint distribution above, what is the marginal distribution for `Rain`? Compute by hand and use the code below to check your work.

In [ ]:
# sum all rows where Rain=True
enumerate_joint(['Forecast','Umbrella'], dict(Rain=True), joint_dist)

## Conditional Probabilities

If we have some evidence that an event occurred, what are the odds that some other event has also occurred? Formally, an **event** is some assignment of variables - complete or incomplete - that can be simultaneously observed. For example: It is forecast to rain. What are the odds that our friend brought her umbrella? We can compute the **conditional probability** as follows:

$$P(a\mid{b}) =  \frac{P(a,b)}{P(b)}$$

For our problem, that would be:

$$P(+umbrella\mid{+forecast}) = \frac{P(+umbrella,+forecast)}{P(+forecast)}$$

Using the respective rows from the joint distribution, we get:

$$\frac{0.25 + 0.14}{0.25 + 0.09 + 0.14 + 0.08} = \frac{0.39}{0.56} = 0.69$$`

In [ ]:
evidence = dict(Forecast=True)
cond_dist = enumerate_joint_ask('Umbrella', evidence, joint_dist)
cond_dist.show_approx()

## Inference by Enumeration 

We used `enumerate_joint_ask` to **query** for the conditional distribution of `Umbrella` given the observed evidence. This strategy is called **inference by enumeration**. It requires a full joint distribution over all variables and does three things:

1. Fixes the **evidence** or **observed** variables. Eliminate all entries from the distribution that don't contain our evidence. 
2. Sum out any **hidden variables** or variables in the remaining entries that are not in the query or evidence. 
3. Normalize.

This is a good strategy, but is $O(d^n)$ in both time and space. That is the problem we'll need to deal with. 

## Product Rule

There are some interesting properties of probability distributions that will come in handy next time. The **product rule** is simply rewriting the definition of conditional probability as:

$$P(b)P(a\mid{b}) = P(a,b)$$

This means that if you have a conditional distribution between $a$ and $b$ and the distribution of $b$, then you also have the full joint distribution over $a$ and $b$.  

## Chain Rule

The **chain rule** takes the product rule one step further. If $P(a,b)$ is $P(b)P(a\mid{b})$, then it is easy to apply the product rule to see that $P(a,b,c)$ is $P(b)P(a\mid{b})P(c\mid{a,b})$. If you replace each conditional probability with the definition of conditional probability, you'll see that things cancel out to give you the full joint distribution. We can generalize this as:

$$P(x_1,x_2,...,x_i) = \prod_i P(x_i|x_1,...,x_{i-1})$$

This means there are $n!$ ways to write the full joint distribution over $x_1,x_2,...,x_i$. 